# Set of tools and function to use Matlab MEOP scripts within python

In [1]:
import meop_process
from pathlib import Path
import sys
import os
import shutil
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import gsw
from IPython.display import display,Image
from importlib import reload
import meop

# Code to process data

In [44]:
meop_process.start_matlab()

matlab started
PWD: /home/jupyter-froqu/GitHub/meop_qc/python_toolbox


In [45]:
conf = meop_process.init_mirounga()

conf = init_mirounga;



In [46]:
EXP = 'ct107'
meop_process.load_info_deployment(EXP=EXP)
meop_process.print_matlab("info_deployment")

conf = init_mirounga;


info_deployment = 

  struct with fields:

               EXP: 'ct107'
                PI: 'COSTA'
            NATION: 'USA'
            nomfic: 'ct107_ODV.txt'
               dir: '/home/jupyter-froqu/MEOP_process/final_dataset_prof/ct107/'
           process: 1
            public: 1
          list_tag: [24x1 struct]
      list_tag_lr0: [24x1 struct]
      list_tag_lr1: [0x1 struct]
      list_tag_hr0: [24x1 struct]
      list_tag_hr1: [1x1 struct]
      list_tag_hr2: [0x1 struct]
      list_tag_fr0: [11x1 struct]
      list_tag_fr1: [0x1 struct]
    list_smru_name: {1x24 cell}




In [48]:
meop_process.process_single_deployment(EXP)

conf = init_mirounga;

remove_deployment(conf,EXP);

create_ncargo(conf,EXP);
Process EXP=ct107
	24 tags
	4902 profiles
  ct107-933-13: 0 profiles and 0 Sprofiles removed
Tag ct107-933b-13 lr0 removed
  ct107-933b-13: 0 profiles and 0 Sprofiles removed
  ct107-934-13-N1: 0 profiles and 0 Sprofiles removed
  ct107-934-13-N2: 0 profiles and 0 Sprofiles removed
  ct107-938-13-N1: 0 profiles and 0 Sprofiles removed
  ct107-938-13-N2: 0 profiles and 129 Sprofiles removed
  ct107-940-13: 0 profiles and 0 Sprofiles removed
  ct107-941-13-N1: 0 profiles and 0 Sprofiles removed
  ct107-941-13-N2: 0 profiles and 234 Sprofiles removed
  ct107-947-13-N1: 0 profiles and 0 Sprofiles removed
  ct107-947-13-N2: 0 profiles and 0 Sprofiles removed
  ct107-950-13-N1: 0 profiles and 0 Sprofiles removed
  ct107-950-13-N2: 0 profiles and 236 Sprofiles removed
  ct107-951-13-N1: 0 profiles and 0 Sprofiles removed
  ct107-951-13-N2: 0 profiles and 0 Sprofiles removed
  ct107-952-13-N1: 0 profiles and 0 Sprofi

True

In [39]:
meop_process.run_command("generate_plot1(conf,EXP);")

generate_plot1(conf,EXP);
Index exceeds the number of array elements (12).

Error in generate_plot1 (line 64)
        strrep(info_deployment.list_tag(itag).name,'_lr0','_lr1'));




False

In [9]:
meop_process.run_command("generate_plot2(conf,EXP);")

In [47]:
one_smru_name='ct107-933-13'
meop_process.process_single_tag(one_smru_name=one_smru_name)
meop_process.run_command("generate_plot1(conf,EXP,one_smru_name);")
meop_process.run_command("generate_plot2(conf,EXP,one_smru_name);")

conf = init_mirounga;

create_ncargo(conf,EXP,one_smru_name);
Process smru_name=ct107-933-13
	24 tags
	4902 profiles
  ct107-933-13: 0 profiles and 0 Sprofiles removed
SMRU ct107-933-13, PTT   133774: use crawl locations

create_fr0(conf,EXP,one_smru_name);
Process hr smru_name=ct107-933-13
  SMRU ct107-933-13, PTT   133774: use crawl locations for fr data
  Date issue in ct107-933-13 hr data, 12933 instr_id: 1 profiles removed
  ct107-933-13: hr tags with 475 profiles
  ct107-933-13: 0 profiles and 0 Sprofiles removed

update_metadata(conf,EXP,one_smru_name);

apply_adjustments(conf,EXP,one_smru_name);

apply_tlc(conf,EXP,one_smru_name);

apply_tlc_fr(conf,EXP,one_smru_name);

create_hr2(conf,EXP,one_smru_name);
Create hr2 version [by default, hr1 --> hr2]
  ct107-933-13: high resolution fr1 --> hr2

generate_plot1(conf,EXP,one_smru_name);
calibration plots: ct107-933-13

generate_plot2(conf,EXP,one_smru_name);
plot diag ct107: lr0, raw
plot diag ct107: lr1, adj
plot diag ct107: hr1, 

True

In [40]:
meop_process.stop_matlab()

In [43]:
meop_process.start_matlab()
print(meop_process.eng)
meop_process.stop_matlab()

matlab started
PWD: /home/jupyter-froqu/GitHub/meop_qc/python_toolbox
